# Gathering Data

In [18]:
from functions import *
from scivol import *
import nibabel as nib
import ipyvolume as ipv
import numpy as np
import json

In [3]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T2w.nii.gz")
output_folder = os.path.join(proj_root, "output/")
mask_seq_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/slice_images_mask"

/Users/joachimpfefferkorn/repos/neurovolume


In [4]:
anat_img = nib.load(mri_input_filepath)
full_anat = create_volume(normalize_array(anat_img.get_fdata()))
brain_mask = build_bool_mask(mask_seq_path, full_anat)
brain_anat = create_masked_normalized_tensor(full_anat, brain_mask)
ext_anat = create_masked_normalized_tensor(full_anat, brain_mask, keep_when=False)
# don't love the inconsistent nesting of these volume creation functions 


bool masks built v1
creating masked normal tensor
creating masked normal tensor


# Quick Visual Check
Quick note on the following `ipv` functions:
They appear to be normalizing the geometry to be a perfect cube. I wonder if there is a way to apply the affine to this?

In [5]:
#ipv.quickvolshow(full_anat)

In [6]:
#ipv.quickvolshow(brain_anat)

In [7]:
#ipv.quickvolshow(ext_anat)

# Building Scivol

In [8]:
affine = anat_img.affine
print(type(affine))

<class 'numpy.ndarray'>


Initialize the scivol object

In [9]:
skullstrip = Scivol("skullstrip", anat_img.affine, 0.2)

Create and add the grids

In [10]:
skullstrip.add_grids([Grid("full_anat",[full_anat]),
Grid("brain_anat", [brain_anat]),
Grid("ext_anat", [ext_anat])])


writing full_anat to grid
writing brain_anat to grid
writing ext_anat to grid


In [11]:
skullstrip.save_scivol(output_folder)

saving skullstrip.scivol
writing skullstrip.scivol


Before moving to blender to render these images, let's make sure that we can parse these scivol files here

In [26]:
scivol_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with open(scivol_path, 'r') as file:
    data = json.load(file)
print("scivol name: ", data['name'], "\n")
for grid_key in data['grids'].keys():
    print(grid_key)
    vol = np.array(data['grids'][grid_key]['frames'][0])

scivol name:  skullstrip 

full_anat
brain_anat
ext_anat


In [38]:
print(data['tolerance'])
print((np.asarray(data['affine'])))

0.2
[[  -0.5           0.           -0.          129.70300293]
 [  -0.            0.5          -0.         -110.68299866]
 [   0.            0.            4.          -74.83930206]
 [   0.            0.            0.            1.        ]]


In [ ]:
print(data['grids'].keys())
for grid in data['grids']:
    print(grid)

dict_keys(['full_anat', 'brain_anat', 'ext_anat'])


Cool, let's quickly visualize this

In [ ]:
ipv.quickvolshow(np.array(data['grids']['full_anat']['frames'][0]))
# only 0 as it's just 3D, no temporal dimension involved here

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

Great! Now lets go build and use a blender plugin